# Megafigures: brightness + gamma (definitive vs cropped_def)

Notebook pensat per fer proves visuals ràpides de com canvien les imatges quan apliques `apply_brightness_and_gamma`.

- Pots triar si vols veure les imatges **senceres** de `definitive/` o les **cropped** que estan **directament** dins `cropped_def/`.
- La idea és que copiïs/enganxis la **cel·la plantilla** i entre cel·les només canviïs `BRIGHTNESS` i `GAMMA` (i, si cal, `SOURCE`).


In [1]:
from __future__ import annotations

from pathlib import Path
import sys

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


def find_project_root(start: Path | None = None, marker_dir: str = "data") -> Path:
    start = Path.cwd() if start is None else Path(start).resolve()
    for p in [start, *start.parents]:
        if (p / marker_dir).exists() and (p / marker_dir).is_dir():
            return p
    raise FileNotFoundError(f"No he trobat '{marker_dir}/' pujant des de {start}")


PROJECT_ROOT = find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.raw_image_treatment import apply_brightness_and_gamma

print("PROJECT_ROOT:", PROJECT_ROOT)


PROJECT_ROOT: c:\Users\david\Desktop\Uni\potato-dry-matter-optics-ml


In [ ]:
# ------------------------------------------------------------
# Config: rutes i utilitats
# ------------------------------------------------------------
DIR_DEFINITIVE = PROJECT_ROOT / "data/input/raw/raw_images/definitive"
DIR_CROPPED_DEF = PROJECT_ROOT / "data/input/processed/cropped_def"

import re

def natural_sort_key(p: Path):
    parts = re.split(r"(\d+)", p.name)
    key = []
    for part in parts:
        key.append(int(part) if part.isdigit() else part.lower())
    return key


def list_image_paths(
    source: str,
    pattern: str = "*.png",
    max_images: int | None = None,
    shuffle: bool = False,
    seed: int = 0,
) -> list[Path]:
    if source == "definitive":
        base_dir = DIR_DEFINITIVE
    elif source == "cropped_def":
        base_dir = DIR_CROPPED_DEF
    else:
        raise ValueError("source ha de ser 'definitive' o 'cropped_def'")

    if not base_dir.exists():
        raise FileNotFoundError(f"No existeix: {base_dir}")

    paths = [p for p in base_dir.glob(pattern) if p.is_file()]
    paths.sort(key=natural_sort_key)

    # IMPORTANT: en cropped_def volem les "soltes" directament dins la carpeta,
    # no les de subcarpetes (glob no baixa per defecte).
    if shuffle:
        rng = np.random.default_rng(seed)
        rng.shuffle(paths)

    if max_images is not None:
        paths = paths[: int(max_images)]

    return paths


def plot_megagrid(images: list[tuple[str, Image.Image]], title: str, n_cols: int = 3, cell_size: float = 5.0) -> None:
    if len(images) == 0:
        print(f"(buit) {title}")
        return

    n_rows = (len(images) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(cell_size * n_cols, cell_size * n_rows))

    axes = np.array(axes).reshape(-1)
    for ax, (name, img) in zip(axes, images):
        ax.imshow(img)
        ax.set_title(name, fontsize=9)
        ax.axis("off")

    for ax in axes[len(images) :]:
        ax.axis("off")

    plt.tight_layout()
    plt.show()


print("definitive exists:", DIR_DEFINITIVE.exists(), "|", DIR_DEFINITIVE)
print("cropped_def exists:", DIR_CROPPED_DEF.exists(), "|", DIR_CROPPED_DEF)


## Cel·la plantilla (copia/enganxa)

Canvia només `BRIGHTNESS` i `GAMMA` (i si vols, `SOURCE`). La resta són opcions per no haver d'editar la lògica cada cop.


In [ ]:
# ==============================
# PARAMETRES (edita aquí només)
# ==============================
BRIGHTNESS = 2.0
GAMMA = 0.8

# 'definitive' o 'cropped_def'
SOURCE = "definitive"

# Patró de fitxers (exemples):
# - definitive: "*.png" o "p3_*.png"
# - cropped_def: "*_cropped.png" o "*.png"
PATTERN = "*.png"

MAX_IMAGES = None  # ex: 24
SHUFFLE = False
SEED = 0

N_COLS = 4
CELL_SIZE = 5.5
# ==============================

paths = list_image_paths(
    SOURCE,
    pattern=PATTERN,
    max_images=MAX_IMAGES,
    shuffle=SHUFFLE,
    seed=SEED,
)

print(f"SOURCE={SOURCE} | pattern={PATTERN} | n={len(paths)}")

processed: list[tuple[str, Image.Image]] = []
for p in paths:
    img = apply_brightness_and_gamma(p, brightness=BRIGHTNESS, gamma=GAMMA)
    processed.append((p.name, img))

plot_megagrid(
    processed,
    title=f"{SOURCE} | brightness={BRIGHTNESS} | gamma={GAMMA} | {PATTERN}",
    n_cols=N_COLS,
    cell_size=CELL_SIZE,
)


## Exemple ràpid (cropped_def)

Això mostra només els PNGs que estan directament a `data/input/processed/cropped_def/` (no subcarpetes).


In [ ]:
BRIGHTNESS = 1
GAMMA = 1
SOURCE = "cropped_def"
PATTERN = "*_cropped.png"
MAX_IMAGES = None
N_COLS = 6

paths = list_image_paths(SOURCE, pattern=PATTERN, max_images=MAX_IMAGES)
processed = [(p.name, apply_brightness_and_gamma(p, brightness=BRIGHTNESS, gamma=GAMMA)) for p in paths]
plot_megagrid(processed, title=f"{SOURCE} | brightness={BRIGHTNESS} | gamma={GAMMA}", n_cols=N_COLS, cell_size=4.5)

In [ ]:
BRIGHTNESS = 1.6
GAMMA = 0.9
SOURCE = "cropped_def"
PATTERN = "*_cropped.png"
MAX_IMAGES = None
N_COLS = 6

paths = list_image_paths(SOURCE, pattern=PATTERN, max_images=MAX_IMAGES)
processed = [(p.name, apply_brightness_and_gamma(p, brightness=BRIGHTNESS, gamma=GAMMA)) for p in paths]
plot_megagrid(processed, title=f"{SOURCE} | brightness={BRIGHTNESS} | gamma={GAMMA}", n_cols=N_COLS, cell_size=4.5)
